In [ ]:
import spacy

In [ ]:
# !python -m spacy download pt_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     --------------------------------------  12.8/13.0 MB 73.2 MB/s eta 0:00:01
     --------------------------------------- 13.0/13.0 MB 50.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
nlp = spacy.load("pt_core_news_sm")

In [122]:
exemplo="""
No Brasil, a popular série "Os Simpsons" é conhecida pela capacidade de refletir sobre questões sociais com uma abordagem crítica e bem-humorada. Em determinado episódio, o personagem Homer se depara com um robô doméstico que, apesar da promessa de facilitar sua vida, acaba complicando-a ainda mas. Esse cenário, embora apresentado de forma cômica, reflete uma preocupação real sobre a integração das tecnologias, especialmente da inteligência artificial (IA), em nossas vidas.
A inteligência artificial, vem sendo amplamente desenvolvida nos últimos tempos, e tem potencial para transformar positivamente diversos aspectos da vida. No entanto, sua adoção generalizada e sem controle levanta questões que necessitam ser cuidadosamente analisadas. Um dos principais desafios é o uso irracional da IA por estudantes, que se tornam dependentes dessas tecnologias para resolver problemas limitando seu senso critico, raciocínio lógico e evolução do processo criativo, ao invés de se empenharem na busca de soluções e no desenvolvimento de novas habilidades.
Esse comportamento não só afeta o aprendizado individual, como também impacta negativamente o futuro profissional desses estudantes. A falta das habilidades práticas na resolução de problemas pode danificar a capacidade de adaptação com novas situações no ambiente de trabalho. Pois se este aluno não é capaz de explicar o processos da IA para chegar em determinada solução, como o tal teria condições de aplicar este conhecimento em situações diversas? assim como explicou o professor Marco Antonio Moreira, no IV Encontro Internacional sobre Aprendizagem, "a aprendizagem significativa só acontece quando o aluno é capaz de explicar um novo conhecimento com suas próprias palavras e mostra um esforço deliberado para aprender, tanto cognitiva quanto afetivamente".
Em suma, embora a inteligência artificial tenha o potencial de trazer muitos benefícios, seu uso inadequado pode comprometer a qualidade da educação e a capacidade profissional dos indivíduos. Para mitigar esses riscos, as escolas podem adotar várias estratégias.
Uma abordagem eficaz seria incentivar a resolução de problemas não apenas por meio de atividades tradicionais,  também por meio de competições e feiras tecnológicas. O apoio do estado e das instituições educacionais para a participação dos alunos em eventos como essas feiras pode ser crucial. Esse envolvimento pode ajudar os estudantes a desenvolver habilidades práticas e a compreender que a tecnologia deve ser vista como uma ferramenta de suporte, e não como uma muleta. O uso irresponsável da tecnologia pode, de fato, complicar ainda mais sua vida assim você pode se tornar um Homer Simpsons."""

In [ ]:
def identificar_erro(frase):
    doc = nlp(frase)
    erros = []

    for i, token in enumerate(doc):
        if token.text.lower() == "mais":
            if i > 0:
                ant = doc[i - 1]
                prox = doc[i + 1] if i + 1 < len(doc) else None
                # Regra 2 - erro: "mais" após vírgula e antes de verbo ou advérbio
                if ant.text == "," and prox and (prox.pos_ == "VERB" or prox.pos_ == "ADV"):
                    erros.append("erro: 'mais' usado como 'mas'")
                else:
                    erros.append("mais de adição")

        elif token.text.lower() == "mas":
            pontuacao = ['.', '!', '?', ';']
            prox = doc[i + 1] if i + 1 < len(doc) else None
            # Regra 1 - erro: 'mas' seguido de substantivo ou adjetivo isolado
            if prox and (prox.pos_ in ["NOUN"]):
                erros.append("erro: 'mas' usado como 'mais'")
            # Regra 3 - erro: 'mas' no final da frase
            elif i == len(doc) - 1 or (prox and prox.text in pontuacao):
                # Só é erro se estiver no final ou antes de pontuação final
                erros.append("erro: 'mas' no final da frase")
            else:
                erros.append("mas de contraste")
    
    # Se houver erros, retornar todos como lista
    if erros:
        return erros

    return ["não identificado"]


In [140]:
testes = [
    "Ele queria sair, mas estava cansado.",# "mas de contraste",  0
    "Ele comprou mais frutas do que ontem.",# "mais de adição",  1
    "Ele comprou mas frutas do que ontem.", # Erro: mas seguido de substantivo  2
    "Ele chegou cedo, mais não ficou muito.", # Erro: mais após vírgula antes verbo deveria ser mas  3
    "Eu gosto de sorvete, mais também de bolo.", #erro "mais de contraste",  4
    "Quero ir, mas.",# "erro obrigatório",  # mas no final, frase incompleta  5
    "Ele saiu, mas feliz.", #"mais de contraste" estranho mas nao está errado  6
    "Ela disse que viria, mais não veio.", #"erro obrigatório"  7
    "Gostaria de mais café, mas não tem.", #"mas de contraste",   8
    "Ele quer mais paz, mais sabe que não vai.",# "mais de adição",  9
    "Ela queria sair, mas o cansaço venceu.",# "mas de contraste",  10 
    "Você trouxe mais, mas não era suficiente.",# "mas de contraste",  11
    "Ele estudou, mais não passou.",# "erro obrigatório",   12
    "Eu gosto de mais, mais quero menos.",# "mais de adição",  13
    "Eu vi, mas.", #"erro obrigatório",  14
    "Eu quero mais.", #"mais de adição",  # Caso sutil, geralmente erro  15
    "O cachorro, mas corre rápido.", #"mas de contraste"  # Pode confundir, mas correto  16
]

In [145]:
frases_pegadinhas = [
    "Ele não só estudou, mas também trabalhou muito para passar no concurso.", # "mas" com sentido de adição/ênfase
    "Não chore mais, por favor.", # "mais" como advérbio de tempo (não mais)
    "Ele tinha mais argumentos, mas preferiu ficar calado.", # "mais" de quantidade, "mas" de oposição
    "Eu queria mais que isso, mas não consegui.", # "mais" de intensidade, "mas" de oposição
    "Mais vale um pássaro na mão do que dois voando.", # Expressão com "mais" (comparativo)
    "Não sou mais um menino, mas um homem feito.", # "mais" de tempo (não sou mais), "mas" de oposição
    "Ela queria mais dinheiro, mas seu salário era baixo.", # "mais" de quantidade, "mas" de oposição
    "Não se preocupe mais com isso, mas sim com o futuro.", # "mais" de tempo, "mas" de oposição/alternativa
    "Eu não tenho mais dúvidas, mas sim certezas.", # "mais" de quantidade (no sentido de 'não ter mais'), "mas" de oposição
    "Ele não tinha mais nada a dizer, mas precisava ir embora.", # "mais" de quantidade, "mas" de oposição
    "Ainda que não goste, mas terei de fazer.", # Embora "mas" aqui possa parecer redundante com "ainda que", em português coloquial é aceitável para reforçar a oposição.
    "Não quero mais problemas, mas sim soluções.", # "mais" de quantidade, "mas" de oposição
    "Eu não vou mais lá, mas ele vai.", # "mais" de tempo, "mas" de oposição
    "Ele queria mais livros, mas a biblioteca estava fechada.", # "mais" de quantidade, "mas" de oposição
    "Ela não é mais criança, mas uma adolescente." # "mais" de tempo, "mas" de oposição
]

In [146]:
for i , iten in enumerate(frases_pegadinhas):
    print(i)
    print(identificar_erro(iten))
    print("-"*50)

0
['mas de contraste']
--------------------------------------------------
1
['mais de adição']
--------------------------------------------------
2
['mais de adição', 'mas de contraste']
--------------------------------------------------
3
['mais de adição', 'mas de contraste']
--------------------------------------------------
4
['não identificado']
--------------------------------------------------
5
['mais de adição', 'mas de contraste']
--------------------------------------------------
6
['mais de adição', 'mas de contraste']
--------------------------------------------------
7
['mais de adição', 'mas de contraste']
--------------------------------------------------
8
['mais de adição', 'mas de contraste']
--------------------------------------------------
9
['mais de adição', 'mas de contraste']
--------------------------------------------------
10
['mas de contraste']
--------------------------------------------------
11
['mais de adição', 'mas de contraste']
-------------------

In [141]:
for i , iten in enumerate(testes):
    print(i)
    print(identificar_erro(iten))
    print("-"*50)

0
['mas de contraste']
--------------------------------------------------
1
['mais de adição']
--------------------------------------------------
2
["erro: 'mas' usado como 'mais'"]
--------------------------------------------------
3
["erro: 'mais' usado como 'mas'"]
--------------------------------------------------
4
["erro: 'mais' usado como 'mas'"]
--------------------------------------------------
5
["erro: 'mas' no final da frase"]
--------------------------------------------------
6
['mas de contraste']
--------------------------------------------------
7
["erro: 'mais' usado como 'mas'"]
--------------------------------------------------
8
['mais de adição', 'mas de contraste']
--------------------------------------------------
9
['mais de adição', "erro: 'mais' usado como 'mas'"]
--------------------------------------------------
10
['mas de contraste']
--------------------------------------------------
11
['mais de adição', 'mas de contraste']
-------------------------------